In [1]:
#Imporing important libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import sklearn.metrics as metrics
import math

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

#Making copr of the train and test dataset
c_test  = df_test.copy()
c_train  = df_train.copy()

In [3]:
#To display first 5 entries
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
#To display first 5 entries
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [5]:
#concatenating Train and test dataset

c_train['train']  = 1
c_test['train']  = 0

df = pd.concat([c_train, c_test], axis=0,sort=False)

df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,train
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0,1
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0,1
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0,1
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0,1
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0,1


In [6]:
#Percentage of null values

df_col_name = df.columns
values = []

for col in df_col_name:
    values.append(((df[col].isnull().sum()/df.shape[0])*100))

df_nan = pd.DataFrame(df_col_name,columns=['Feature'])
df_nan['percent'] = values 
df_nan

,Feature,percent
0,Id,0.000000
1,MSSubClass,0.000000
2,MSZoning,0.137033
3,LotFrontage,16.649538
4,LotArea,0.000000
...,...,...
77,YrSold,0.000000
78,SaleType,0.034258
79,SaleCondition,0.000000
80,SalePrice,49.982871


In [7]:
#No. of features with more than 50% of missing values 

df_nan = df_nan[df_nan.percent > 50]
df_nan.sort_values("percent", ascending=False)

,Feature,percent
72,PoolQC,99.657417
74,MiscFeature,96.402878
6,Alley,93.216855
73,Fence,80.438506


In [8]:
#Droping columns with more than 50% of missing values 

df = df.drop(['Alley','PoolQC','Fence','MiscFeature'],axis=1)

In [9]:
#Separating object and numerical feature

object_columns_df = df.select_dtypes(include=['object'])
numerical_columns_df =df.select_dtypes(exclude=['object'])

In [10]:
#Number of null values in each feature

null_counts = object_columns_df.isnull().sum()
null_counts

MSZoning            4
Street              0
LotShape            0
LandContour         0
Utilities           2
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
RoofStyle           0
RoofMatl            0
Exterior1st         1
Exterior2nd         1
MasVnrType         24
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           81
BsmtCond           82
BsmtExposure       82
BsmtFinType1       79
BsmtFinType2       80
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
KitchenQual         1
Functional          2
FireplaceQu      1420
GarageType        157
GarageFinish      159
GarageQual        159
GarageCond        159
PavedDrive          0
SaleType            1
SaleCondition       0
dtype: int64

In [11]:
#Filling missing values with 'none'
none_col = ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','GarageType','GarageFinish','GarageQual',
            'FireplaceQu','GarageCond']
object_columns_df[none_col]= object_columns_df[none_col].fillna('None')

#Filling missing value with mode
object_columns_df = object_columns_df.fillna('mode()')

In [12]:
#Number of null values in each feature

null_counts = numerical_columns_df.isnull().sum()
null_counts

Id                  0
MSSubClass          0
LotFrontage       486
LotArea             0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
MasVnrArea         23
BsmtFinSF1          1
BsmtFinSF2          1
BsmtUnfSF           1
TotalBsmtSF         1
1stFlrSF            0
2ndFlrSF            0
LowQualFinSF        0
GrLivArea           0
BsmtFullBath        2
BsmtHalfBath        2
FullBath            0
HalfBath            0
BedroomAbvGr        0
KitchenAbvGr        0
TotRmsAbvGrd        0
Fireplaces          0
GarageYrBlt       159
GarageCars          1
GarageArea          1
WoodDeckSF          0
OpenPorchSF         0
EnclosedPorch       0
3SsnPorch           0
ScreenPorch         0
PoolArea            0
MiscVal             0
MoSold              0
YrSold              0
SalePrice        1459
train               0
dtype: int64

In [13]:
#Filling missing values with forward_fill

numerical_columns_df['LotFrontage'] = numerical_columns_df['LotFrontage'].fillna(method='ffill')

#Filling missing values of 'GarageYrBlt' with 'YrSold'

numerical_columns_df['GarageYrBlt'].fillna(numerical_columns_df['YrSold'],inplace=True)

#Filling missing values with zero

numerical_columns_df.fillna(0,inplace=True)

In [14]:
#Creating new column 'Age_House'

numerical_columns_df['Age_House']= (numerical_columns_df['YrSold']-numerical_columns_df['YearBuilt'])

In [15]:
#To find if any negative value in Age_House

numerical_columns_df[numerical_columns_df['Age_House'] < 0]

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,train,Age_House
1089,2550,20,128.0,39290,10,5,2008,2009,1224.0,4010.0,...,0,0,0,0,17000,10,2007,0.0,0,-1


In [16]:
#Since the Age_House is -1 filling its value with 2009

numerical_columns_df.loc[numerical_columns_df['YrSold'] < numerical_columns_df['YearBuilt'],'YrSold' ] = 2009
numerical_columns_df['Age_House']= (numerical_columns_df['YrSold']-numerical_columns_df['YearBuilt'])

In [17]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

le = LabelEncoder()

In [18]:
#Label encoding the object feature

for col in object_columns_df.columns.tolist():
    object_columns_df[col] = le.fit_transform(object_columns_df[col])

In [19]:
#Concatenating object and numberical features

df_final = pd.concat([object_columns_df, numerical_columns_df], axis=1,sort=False)
df_final.head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,train,Age_House
0,3,1,3,3,0,4,0,5,2,2,...,0,0,0,0,0,2,2008,208500.0,1,5
1,3,1,3,3,0,2,0,24,1,2,...,0,0,0,0,0,5,2007,181500.0,1,31
2,3,1,0,3,0,4,0,5,2,2,...,0,0,0,0,0,9,2008,223500.0,1,7
3,3,1,0,3,0,0,0,6,2,2,...,272,0,0,0,0,2,2006,140000.0,1,91
4,3,1,0,3,0,2,0,15,2,2,...,0,0,0,0,0,12,2008,250000.0,1,8


In [20]:
#Droping column 'Id'
df_final = df_final.drop(['Id',],axis=1)

#Separating train dataset and droping train column
df_train = df_final[df_final['train'] == 1]
df_train = df_train.drop(['train',],axis=1)

#Separating test dataset and droping train and SalePrice column
df_test = df_final[df_final['train'] == 0]
df_test = df_test.drop(['SalePrice'],axis=1)
df_test = df_test.drop(['train',],axis=1)

In [21]:
#Separting df_train in independent and dependent variable
X=df_train.drop(['SalePrice'],axis=1)
y=df_train['SalePrice']

In [22]:
#Splitting df_train in train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

#### Building model with Linear Regression:

In [23]:
# Importing LinearRegression from sklearn
from sklearn.linear_model import LinearRegression

# Creating object and fitting the model
lin_reg = LinearRegression()
model = lin_reg.fit(X_train,y_train)


In [24]:
# Predicting for test dataset
y_pred = model.predict(X_test)

In [25]:
# Creating Actual and Predicted dataset 
df1 = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df1

,Actual,Predicted
258,231500.0,208149.741240
267,179500.0,184630.487929
288,122000.0,97162.344638
649,84500.0,71556.437837
1233,142000.0,131636.698251
...,...,...
793,225000.0,248936.071039
631,209500.0,218447.205906
1172,171900.0,170393.163958
1167,173000.0,183749.281953


In [26]:
# Model Evaluation 

# Importing metrics from sklearn 
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

# To find Mean Absolute Error(mse)
mse = (mean_absolute_error(y_test, y_pred))
print("MAE:",mse)

# To find Root Mean Squared Error(rmse)
rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))
print("RMSE:",rmse)

# To find coefficient of determination
r2 =  r2_score(y_test, y_pred)
print("R-Square:",r2)

MAE: 21103.568070053094
RMSE: 34645.568140610885
R-Square: 0.8318957302519518


#### Using XGBoost and LGBM:

In [27]:
xgb =XGBRegressor( booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=4, min_child_weight=1.5, n_estimators=2400,
             n_jobs=1, nthread=None, objective='reg:linear',
             reg_alpha=0.6, reg_lambda=0.6, scale_pos_weight=1, 
             silent=None, subsample=0.8, verbosity=1)

xgb.fit(X_train, y_train)


[22:45:35] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(colsample_bytree=0.6, learning_rate=0.01, max_depth=4,
             min_child_weight=1.5, n_estimators=2400, reg_alpha=0.6,
             reg_lambda=0.6, subsample=0.8)

In [28]:
predict1 = xgb.predict(X_test)

print('RMSE using XGBoost: ',math.sqrt(metrics.mean_squared_error(y_test, predict1)))

RMSE using XGBoost:  23511.67693768816


In [29]:
lgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=12000, 
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.4, 
                                       )

lgbm.fit(X_train, y_train,eval_metric='rmse')


LGBMRegressor(bagging_fraction=0.75, bagging_freq=5, bagging_seed=7,
              feature_fraction=0.4, learning_rate=0.01, max_bin=200,
              n_estimators=12000, num_leaves=4, objective='regression')

In [30]:
predict2 = lgbm.predict(X_test)

print('RMSE using LGBM: ',math.sqrt(metrics.mean_squared_error(y_test, predict2)))

RMSE using LGBM:  25771.55247760339


In [31]:
xgb.fit(X, y)
lgbm.fit(X, y,eval_metric='rmse')

[22:45:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


LGBMRegressor(bagging_fraction=0.75, bagging_freq=5, bagging_seed=7,
              feature_fraction=0.4, learning_rate=0.01, max_bin=200,
              n_estimators=12000, num_leaves=4, objective='regression')

In [32]:
predict3 = lgbm.predict(df_test)
predict4 = xgb.predict(df_test)

In [33]:
#Combining LGBM and XGBoost
predict5 = ( predict4*0.45 + predict3*0.55)

In [34]:
#To compare the predicted value from all the three predict
pd.DataFrame({'predict3' : predict3,'predict4' : predict4,'predict5' : predict5 },
             columns=['predict3','predict4', 'predict5'])

,predict3,predict4,predict5
0,122316.864777,128717.882812,125197.322502
1,171855.572789,162519.234375,167654.221284
2,181973.807212,181415.343750,181722.500216
3,193188.973551,193227.187500,193206.169828
4,182091.252455,183419.812500,182689.102913
...,...,...,...
1454,83160.929432,81457.281250,82394.288532
1455,77627.144528,79739.796875,78577.835741
1456,169824.315697,162110.343750,166353.029883
1457,124351.263531,118689.781250,121803.593380


In [35]:
df_test = pd.read_csv('test.csv')

In [36]:
#To csv
submission = pd.DataFrame({
        "Id": df_test["Id"],
        "SalePrice": predict3
    })
submission.to_csv('submission3.csv', index=False)

In [37]:
#To csv
submission = pd.DataFrame({
        "Id": df_test["Id"],
        "SalePrice": predict4
    })
submission.to_csv('submission4.csv', index=False)

In [38]:
#To csv
submission = pd.DataFrame({
        "Id": df_test["Id"],
        "SalePrice": predict5
    })
submission.to_csv('submission5.csv', index=False)